Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, ReLU
import os


Load Dataset

In [2]:
dataset_path = os.path.join(os.path.expanduser("~"), "Desktop", "Image classification", "Dataset")

img_height, img_width = 128, 128  # resize images
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 320 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


Build Model

In [4]:
input_layer = Input(shape=(img_height, img_width, 3))
x1 = Conv2D(32, (3,3), padding='same')(input_layer)
x1=ReLU()(x1)
x1=MaxPooling2D((2,2))(x1)

x2 = Conv2D(32, (5,5), padding='same')(input_layer)
x2 = ReLU()(x2)
x2 = MaxPooling2D((2,2))(x2)

merged = Concatenate() ([x1,x2])
merged = ReLU()(merged)
merged = MaxPooling2D((2,2))(merged)

# Another Conv -> ReLU -> Maxpool
x = Conv2D(64, (3,3), padding='same')(merged)
x = ReLU()(x)
x = MaxPooling2D((2,2))(x)

# Flatten + Dense
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(train_data.num_classes, activation='softmax')(x)

# Build model
model = Model(inputs=input_layer, outputs=output_layer)


Compile model

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │        896 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │      2,432 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 128, 128,  │          0 │ conv2d_2[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 128, 128,  │          0 │ conv2d_3[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ re_lu_2[0][0]     │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 64, 64,    │          0 │ re_lu_3[0][0]     │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64, 64,    │          0 │ max_pooling2d_1[… │
│ (Concatenate)       │ 64)               │            │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 64, 64,    │          0 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 32, 32,    │          0 │ re_lu_4[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     36,928 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_5 (ReLU)      │ (None, 32, 32,    │          0 │ conv2d_4[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 16, 16,    │          0 │ re_lu_5[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 16384)     │          0 │ max_pooling2d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │  2,097,280 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 4)         │        516 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,138,052 (8.16 MB)

 Trainable params: 2,138,052 (8.16 MB)

 Non-trainable params: 0 (0.00 B)

Train Model

In [13]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 476ms/step - accuracy: 1.0000 - loss: 0.0269 - val_accuracy: 0.5750 - val_loss: 2.0256
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 388ms/step - accuracy: 1.0000 - loss: 0.0166 - val_accuracy: 0.5875 - val_loss: 2.0459
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 384ms/step - accuracy: 1.0000 - loss: 0.0117 - val_accuracy: 0.5875 - val_loss: 2.2353
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 375ms/step - accuracy: 1.0000 - loss: 0.0077 - val_accuracy: 0.6000 - val_loss: 2.2819
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 375ms/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 0.6125 - val_loss: 2.4622
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 370ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.6125 - val_loss: 2.5057
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 420ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.6125 - val_loss: 2.5717
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 403ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.

Model Accuracy


In [14]:
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy: {val_acc*100:.2f}%")


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 333ms/step - accuracy: 0.6125 - loss: 2.6967
Validation Accuracy: 61.25%
